In [1]:
"""
@author: pho
"""
## Automatic debugger on exception raised
# %pdb on
%pdb off
%load_ext autoreload
%autoreload 2

import sys
import importlib
from pathlib import Path
import numpy as np
import pandas as pd
from benedict import benedict # https://github.com/fabiocaccamo/python-benedict#usage
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only

Automatic pdb calling has been turned OFF


In [12]:
# local_session_path = Path(r'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15')
# local_session_path = Path(r'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15')
local_session_path = Path(r'W:\Data\KDIBA\gor01\one\2006-6-13_14-42-6')

session_stem = local_session_path.stem # '2006-6-08_14-26-15'
session_xml_filepath = local_session_path.joinpath(session_stem).with_suffix('.xml')
print(f'local_session_path: {local_session_path}')
print(f'session_xml_filepath: {session_xml_filepath}')
assert session_xml_filepath.exists() and session_xml_filepath.is_file()

local_session_path: W:\Data\KDIBA\gor01\one\2006-6-13_14-42-6
session_xml_filepath: W:\Data\KDIBA\gor01\one\2006-6-13_14-42-6\2006-6-13_14-42-6.xml


In [13]:
from neuropy.utils.load_exported import LoadXml
out_xml_dict, d = LoadXml(session_xml_filepath)

In [14]:
nChannels = out_xml_dict['nChannels']
print(f'nChannels: {nChannels}')

nChannels: 96


In [8]:
df = out_xml_dict['AnatGrps_df']
# df = out_xml_dict['SpkGrps_df']
# out_xml_dict
df

,originalGroupIdx,groupIdx,channelIDX,splitIdx
0,0,0,72,0
1,0,0,73,0
2,0,0,74,0
3,0,0,75,0
4,0,0,76,0
...,...,...,...,...
91,8,11,67,0
92,8,11,68,0
93,8,11,69,0
94,8,11,70,0


In [ ]:
{k:len(v) for k,v in df.items()}

In [ ]:
np.array(df['splitIdx']).flatten()

In [ ]:
# create a new empty instance
# d = benedict()

# or cast an existing dict
# d = benedict(existing_dict)

# or create from data source (filepath, url or data-string) in a supported format:
# Base64, CSV, JSON, TOML, XML, YAML, query-string
# d = benedict('https://localhost:8000/data.json', format='json')
d = benedict(str(session_xml_filepath.resolve()), format='xml')#['parameters']

# # or in a Django view
# params = benedict(request.GET.items())
# page = params.get_int('page', 1)

In [ ]:
df[df.splitIdx==1]

In [ ]:
out_xml_dict['SpkGrps_ExtendedDf'].originalGroupIdx 

out_xml_dict['SpkGrps_ExtendedDf'].splitIdx

# df['new_col'] = list(zip(df.lat, df.long))

In [ ]:
df['groupIdx'] = list(zip(df.originalGroupIdx , df.splitIdx))
# uniqueGroupSplitPairs = list(zip(df.originalGroupIdx.to_numpy() , df.splitIdx.to_numpy()))
print(f'uniqueGroupSplitPairs: {df.groupIdx}')
unique_groupPairs, indices = np.unique(df.groupIdx, return_inverse=True)
# unique_groupPairs, indices = np.unique(df.groupIdx, return_index=True)
print(f'unique_groupPairs: {unique_groupPairs}, indices: {indices}')
df['groupIdx'] = indices

In [ ]:
df['groupIdx'] = indices
df


# groupIdx = np.arange(len(unique_groupPairs))

In [ ]:
print_keys_if_possible('d', d)

In [ ]:
d.keypaths()

In [ ]:
d.get('parameters', None)

In [ ]:
'parameters.neuroscope'

In [ ]:
d['parameters.anatomicalDescription.channelGroups.group']
# [{'channel': [{'@skip': '0', '#text': '72'},
#    {'@skip': '0', '#text': '73'},
#    {'@skip': '0', '#text': '74'},
#    {'@skip': '0', '#text': '75'},
#    {'@skip': '0', '#text': '76'},
#    {'@skip': '0', '#text': '77'},
#    {'@skip': '0', '#text': '78'},
#    {'@skip': '0', '#text': '79'}]},
#  {'channel': [{'@skip': '0', '#text': '80'},
#    {'@skip': '0', '#text': '81'},
#    {'@skip': '0', '#text': '82'},
#    {'@skip': '0', '#text': '83'},
#    {'@skip': '0', '#text': '84'},
#    {'@skip': '0', '#text': '85'},
#    {'@skip': '0', '#text': '86'},
#    {'@skip': '0', '#text': '87'}]},

In [ ]:
def LoadXml(session_xml_filepath, debug_print=False):
    d = benedict(str(session_xml_filepath.resolve()), format='xml')
    out_xml_dict = {'nBits': int(d['parameters.acquisitionSystem.nBits']), 'nChannels': int(d['parameters.acquisitionSystem.nChannels']), 'samplingRate': int(d['parameters.acquisitionSystem.samplingRate']), 'voltageRange': int(d['parameters.acquisitionSystem.voltageRange']),
 'amplification': int(d['parameters.acquisitionSystem.amplification']), 'offset': int(d['parameters.acquisitionSystem.offset'])}
    out_xml_dict = out_xml_dict | {'Date':d['parameters.generalInfo.date']}
    out_xml_dict = out_xml_dict | {'lfpSampleRate':int(d['parameters.fieldPotentials.lfpSamplingRate'])}
    
    MAX_CHANNEL_GROUP_LENGTH = 8
    ## Processing Channel Groups:
    out_channelGroupsList = []
    # [{'channel': [{'@skip': '0', '#text': '72'},
    #    {'@skip': '0', '#text': '73'},
    #    {'@skip': '0', '#text': '74'},
    #    {'@skip': '0', '#text': '75'},
    #    {'@skip': '0', '#text': '76'},
    #    {'@skip': '0', '#text': '77'},
    #    {'@skip': '0', '#text': '78'},
    #    {'@skip': '0', '#text': '79'}]},
    #  {'channel': [{'@skip': '0', '#text': '80'},
    #    {'@skip': '0', '#text': '81'},
    #    {'@skip': '0', '#text': '82'},
    #    {'@skip': '0', '#text': '83'},
    #    {'@skip': '0', '#text': '84'},
    #    {'@skip': '0', '#text': '85'},
    #    {'@skip': '0', '#text': '86'},
    #    {'@skip': '0', '#text': '87'}]},
    #  ...
    # ]
    for a_group_dict in d['parameters.anatomicalDescription.channelGroups.group']:
        """ a_channel_list: [{'@skip': '0', '#text': '72'}, {'@skip': '0', '#text': '73'}, {'@skip': '0', '#text': '74'}, {'@skip': '0', '#text': '75'}, {'@skip': '0', '#text': '76'}, {'@skip': '0', '#text': '77'}, {'@skip': '0', '#text': '78'}, {'@skip': '0', '#text': '79'}] """
        if debug_print:
            print(a_group_dict['channel'])
        a_channel_list = [int(a_channel_entry['#text']) for a_channel_entry in a_group_dict['channel'] if a_channel_entry['@skip']=='0']
        if debug_print:
            print(a_channel_list)
        if len(a_channel_list) > MAX_CHANNEL_GROUP_LENGTH:
            # split long groups
            if debug_print:
                print(f'MAX_CHANNEL_GROUP_LENGTH: {MAX_CHANNEL_GROUP_LENGTH}')
            chunked_channel_lists = [a_channel_list[i:i+MAX_CHANNEL_GROUP_LENGTH] for i in range(0, len(a_channel_list), MAX_CHANNEL_GROUP_LENGTH)]
            # chunked_channel_lists = [list(array) for array in np.array_split(np.array(a_channel_list), MAX_CHANNEL_GROUP_LENGTH)]
            out_channelGroupsList.extend(chunked_channel_lists)
            if debug_print:
                print(f'split list: {a_channel_list} -> {chunked_channel_lists}')
        else:
            out_channelGroupsList.append(a_channel_list)

    out_xml_dict['ElecGp'] = out_channelGroupsList
    out_xml_dict['nElecGps'] = len(out_channelGroupsList)
    
    
    ## Processing Spike Groups:
    out_spikeGroupsList = []
    # [{'channels': {'channel': ['0', '1', '2', '3', '4', '5', '6', '7']},
    #   'nSamples': '54',
    #   'peakSampleIndex': '26',
    #   'nFeatures': '3'},
    #  {'channels': {'channel': ['8', '9', '10', '11', '12', '13', '14', '15']},
    #   'nSamples': '54',
    #   'peakSampleIndex': '26',
    #   'nFeatures': '3'}, ...

    for a_group_dict in d['parameters.spikeDetection.channelGroups.group']:
        """ a_channel_list: {'channels': {'channel': ['0', '1', '2', '3', '4', '5', '6', '7']}, 'nSamples': '54', 'peakSampleIndex': '26', 'nFeatures': '3'} """
        if debug_print:
            print(a_group_dict)
            print(a_group_dict['channels']['channel']) # ['0', '1', '2', '3', '4', '5', '6', '7']
        a_channel_list = [int(a_channel_entry) for a_channel_entry in a_group_dict['channels']['channel']]
        if debug_print:
            print(a_channel_list)
        if len(a_channel_list) > MAX_CHANNEL_GROUP_LENGTH:
            # split long groups
            if debug_print:
                print(f'MAX_CHANNEL_GROUP_LENGTH: {MAX_CHANNEL_GROUP_LENGTH}')
            chunked_channel_lists = [a_channel_list[i:i+MAX_CHANNEL_GROUP_LENGTH] for i in range(0, len(a_channel_list), MAX_CHANNEL_GROUP_LENGTH)]
            out_spikeGroupsList.extend(chunked_channel_lists)
            if debug_print:
                print(f'split list: {a_channel_list} -> {chunked_channel_lists}')
        else:
            out_spikeGroupsList.append(a_channel_list)

    out_xml_dict['SpkGrps'] = out_spikeGroupsList
    
    return out_xml_dict, d

out_xml_dict, d = LoadXml(session_xml_filepath)
out_xml_dict

In [ ]:


# nElecGps

debug_print = True

        
out_spikeGroupsList

In [ ]:
out_xml_dict = {'nBits': int(d['parameters.acquisitionSystem.nBits']), 'nChannels': int(d['parameters.acquisitionSystem.nChannels']), 'samplingRate': int(d['parameters.acquisitionSystem.samplingRate']), 'voltageRange': int(d['parameters.acquisitionSystem.voltageRange']),
 'amplification': int(d['parameters.acquisitionSystem.amplification']), 'offset': int(d['parameters.acquisitionSystem.offset'])}

In [ ]:
out_xml_dict = out_xml_dict | {'Date':d['parameters.generalInfo.date']}
out_xml_dict

In [ ]:
out_xml_dict = out_xml_dict | {'lfpSampleRate':int(d['parameters.fieldPotentials.lfpSamplingRate'])}
out_xml_dict

In [ ]:
out_channelGroupsList = []
MAX_CHANNEL_GROUP_LENGTH = 8
for a_group_dict in d['parameters.anatomicalDescription.channelGroups.group']:
    """ a_channel_list: [{'@skip': '0', '#text': '72'}, {'@skip': '0', '#text': '73'}, {'@skip': '0', '#text': '74'}, {'@skip': '0', '#text': '75'}, {'@skip': '0', '#text': '76'}, {'@skip': '0', '#text': '77'}, {'@skip': '0', '#text': '78'}, {'@skip': '0', '#text': '79'}] """
    # print(a_group_dict['channel'])
    a_channel_list = [int(a_channel_entry['#text']) for a_channel_entry in a_group_dict['channel'] if a_channel_entry['@skip']=='0']
    print(a_channel_list)
    if len(a_channel_list) > MAX_CHANNEL_GROUP_LENGTH:
        # split long groups
        # print(f'MAX_CHANNEL_GROUP_LENGTH: {MAX_CHANNEL_GROUP_LENGTH}')
        chunked_channel_lists = [a_channel_list[i:i+MAX_CHANNEL_GROUP_LENGTH] for i in range(0, len(a_channel_list), MAX_CHANNEL_GROUP_LENGTH)]
        # chunked_channel_lists = [list(array) for array in np.array_split(np.array(a_channel_list), MAX_CHANNEL_GROUP_LENGTH)]
        out_channelGroupsList.extend(chunked_channel_lists)
        # print(chunked_list)# Returns: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        # print(f'split list: {a_channel_list} -> {chunked_channel_lists}')
    else:
        out_channelGroupsList.append(a_channel_list)
# [{'channel': [{'@skip': '0', '#text': '72'},
#    {'@skip': '0', '#text': '73'},
#    {'@skip': '0', '#text': '74'},
#    {'@skip': '0', '#text': '75'},
#    {'@skip': '0', '#text': '76'},
#    {'@skip': '0', '#text': '77'},
#    {'@skip': '0', '#text': '78'},
#    {'@skip': '0', '#text': '79'}]},
#  {'channel': [{'@skip': '0', '#text': '80'},
#    {'@skip': '0', '#text': '81'},
#    {'@skip': '0', '#text': '82'},
#    {'@skip': '0', '#text': '83'},
#    {'@skip': '0', '#text': '84'},
#    {'@skip': '0', '#text': '85'},
#    {'@skip': '0', '#text': '86'},
#    {'@skip': '0', '#text': '87'}]},
#  ...
# ]

out_channelGroupsList

In [ ]:
d['parameters.acquisitionSystem'] # {'nBits': '16', 'nChannels': '96', 'samplingRate': '32552', 'voltageRange': '20', 'amplification': '1000', 'offset': '0'}

In [ ]:
int(d['parameters.acquisitionSystem']['nChannels'])

# float(a)

In [ ]:
out_xml_dict['AnatGrps']

In [ ]:
d = {
    "out_xml_dict": out_xml_dict,
}

In [ ]:
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.External.pyqtgraph.Qt import QtCore, QtGui 
from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_DataTreeWidget import plot_dataTreeWidget
tree, app = plot_dataTreeWidget(data=out_xml_dict, title='PhoOutputDataTreeApp')
tree.show()
pg.exec() # required in an empty notebook to get the window to show instead of just launching and locking up